In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
num_user_ratings=[URM_train[u].nnz for u in range(URM_train.shape[0])]

In [5]:
threshold=10

In [6]:
URM_train_pop=URM_train[[u for u in range(URM_train.shape[0]) if num_user_ratings[u]>=threshold]]
URM_valid_pop=URM_valid[[u for u in range(URM_valid.shape[0]) if num_user_ratings[u]>=threshold]]
URM_train_unpop=URM_train[[u for u in range(URM_train.shape[0]) if num_user_ratings[u]<threshold and num_user_ratings[u]!=0]]
URM_valid_unpop=URM_valid[[u for u in range(URM_valid.shape[0]) if num_user_ratings[u]<threshold and num_user_ratings[u]!=0]]

In [7]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
RP3brecommender= RP3betaRecommender(URM_train_pop)
RP3brecommender.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

RP3betaRecommender: URM Detected 5072 (19.53 %) cold items.


In [8]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
ItemKNNrecommender=ItemKNNCBFRecommender(URM_train_pop, ICM_all)
ItemKNNrecommender.fit(topK=393, shrink=6, similarity="dice")

ItemKNNCBFRecommender: URM Detected 5072 (19.53 %) cold items.
Similarity column 25975 ( 100 % ), 3803.57 column/sec, elapsed time 0.11 min


In [9]:
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
smallHybridrecommender= ItemKNNSimilarityHybridRecommender(URM_train_pop, RP3brecommender.W_sparse, ItemKNNrecommender.W_sparse)
smallHybridrecommender.fit(topK=1100, alpha=0.5)

ItemKNNSimilarityHybridRecommender: URM Detected 5072 (19.53 %) cold items.


In [10]:
from MatrixFactorization.IALSRecommender import IALSRecommender
IALSrecommender= IALSRecommender(URM_train_pop)
IALSrecommender.fit(alpha = 49.99999999999999, epochs=10, num_factors=200, confidence_scaling="linear",epsilon=0.002936450234353087, reg=0.01)

IALSRecommender: URM Detected 5072 (19.53 %) cold items.
IALSRecommender: Epoch 1 of 10. Elapsed time 13.52 sec
IALSRecommender: Epoch 2 of 10. Elapsed time 28.02 sec
IALSRecommender: Epoch 3 of 10. Elapsed time 42.41 sec
IALSRecommender: Epoch 4 of 10. Elapsed time 56.87 sec
IALSRecommender: Epoch 5 of 10. Elapsed time 1.19 min
IALSRecommender: Epoch 6 of 10. Elapsed time 1.43 min
IALSRecommender: Epoch 7 of 10. Elapsed time 1.69 min
IALSRecommender: Epoch 8 of 10. Elapsed time 1.94 min
IALSRecommender: Epoch 9 of 10. Elapsed time 2.19 min
IALSRecommender: Epoch 10 of 10. Elapsed time 2.43 min
IALSRecommender: Terminating at epoch 10. Elapsed time 2.43 min


In [11]:
from Base.NonPersonalizedRecommender import TopPop
TopPoprecommender= TopPop(URM_train_pop)
TopPoprecommender.fit()

TopPopRecommender: URM Detected 5072 (19.53 %) cold items.


In [23]:
def WeightGenerator(length,total_sum=100):
    if length == 1:
        yield (total_sum,)
    else:
        for value in range(0,total_sum + 1, 10):
            for permutation in WeightGenerator(length - 1, total_sum - value):
                yield (value,) + permutation

In [24]:
popRec=[smallHybridrecommender,IALSrecommender,TopPoprecommender]
unpopRec=[smallHybridrecommender,IALSrecommender,TopPoprecommender]

In [25]:
from KNN.SimpleWeightedHybridRecommender import SimpleWeightedHybridRecommender

recommender=SimpleWeightedHybridRecommender(URM_train_pop, popRec)
evaluator_validation = EvaluatorHoldout(URM_valid_pop, cutoff_list=[10])
resultDF= pd.DataFrame({"weights":[],"MAP":[]})

for weights in list(WeightGenerator(len(popRec))):
        recommender.fit(np.array(weights)/100)
        MAP,_=evaluator_validation.evaluateRecommender(recommender)
        MAP=MAP[10]["MAP"]
        resultDF=resultDF.append({"weights":[weights],"MAP":[MAP]},ignore_index=True)
        print(weights, MAP)

SimpleWeightedHybridRecommender: URM Detected 5072 (19.53 %) cold items.
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.67 sec. Users per second: 734
(0, 0, 100) 0.010100053445179233
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.76 sec. Users per second: 710
(0, 10, 90) 0.01011589829143738
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.84 sec. Users per second: 690
(0, 20, 80) 0.01011589829143738
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.89 sec. Users per second: 678
(0, 30, 70) 0.01011589829143738
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.83 sec. Users per second: 693
(0, 40, 60) 0.01011589829143738
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.81 sec. Users per second: 698
(0, 50, 50) 0.0101252472630505
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.68 sec. Users per second: 732
(0, 60, 40) 0.010140862608854526
EvaluatorHoldout: Processed 1961 ( 100.00% ) in 2.75 sec. Users per second: 713
(0, 70, 30) 0.01023439785569166
EvaluatorHoldout: Processed 19

In [26]:
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Weighted_Hybrid_popular_results_sorted.csv")